In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import pingouin as pg
import plotly.express as px
import seaborn as sns 
from statannot import add_stat_annotation
import os
import copy
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

d:\Tianyu\software\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
d:\Tianyu\software\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [ ]:
## savgol filter ##
def SG_fil(data,window,column_name,order=2):
    df = data.copy()
    for n in column_name:
        df[n] = savgol_filter(df.loc[:,n],window,order)
    return df

def lumin2cogload(lumin):
    # coeff_left = [2.451072052, -15.47037207, 2.49154129]
    # coeff_right = [2.489320955, -13.62467234, 2.540344354]
    coeff_left = [1.326173, -4.42176, 3.016348]
    coeff_right = [1.225595, -5.16006, 3.3087083]
    cogload_adjust_left = coeff_left[0]*np.exp(coeff_left[1]*lumin) + coeff_left[2]
    cogload_adjust_right = coeff_right[0]*np.exp(coeff_right[1]*lumin) + coeff_right[2]
    return cogload_adjust_left, cogload_adjust_right

In [ ]:
path = '../data/test/'

data_raw = []
for home, dirs, files in os.walk(path):
        for filename in files:
            t = pd.read_csv(home+"/"+filename,index_col=0)
            t = t[90:-1].reset_index()
            t[["participant_id"]] = filename[1:3]
            t[["condition"]] = filename[4:8]

            t['LeftPupilD'][t['LeftPupilD']<0] = np.nan
            t['RightPupilD'][t['RightPupilD']<0] = np.nan
            t[['LeftPupilD','RightPupilD']] = t[['LeftPupilD','RightPupilD']].interpolate(method='linear',axis=0,limit_direction ='both')
            t = SG_fil(t,91,['LeftPupilD','RightPupilD','Lumin'])

            (t[['cogload_adjust_left']], t[['cogload_adjust_right']]) = lumin2cogload(t[['Lumin']])
            t[['cogload_left']] = t[['LeftPupilD']].values - t[['cogload_adjust_left']].values
            t[['cogload_right']] = t[['RightPupilD']].values - t[['cogload_adjust_right']].values

            time_all = len(t)/90